Notebook to extract location coordinate of all MRT/LRT Stations & Bus Stops


In [1]:
import geopandas as gpd
import pandas as pd
import json

# Train Station

- Reading from static file uploaded in LTA Data MAll

In [2]:
fp = 'static_data/TrainStation_Feb2023/RapidTransitSystemStation.shp'
data = gpd.read_file(fp)
data = data.to_crs(epsg=4326)

In [3]:
# Retrueve the latitude and longitude
data['Latitude'] = data.apply(lambda x:x.geometry.centroid.x,axis=1)
data['Longitude'] = data.apply(lambda x:x.geometry.centroid.y,axis=1)

In [4]:
# Remove duplicates & rename columns
data = data.drop_duplicates(subset=['STN_NAM_DE'])
data['Latlng'] = data.apply(lambda x: f"{x['Latitude']},{x['Longitude']}",axis=1)
data.rename(columns={'STN_NAM_DE':'Station Name'},inplace=True)
data.drop(columns=['TYP_CD','STN_NAM','geometry'],inplace=True)
data = pd.DataFrame(data)

In [5]:
results_list = []
for idx, row in data.iterrows():
    results_list.append(row.to_dict())

In [6]:
json_dict = json.dumps(results_list)

In [7]:
with open ('static_data/TrainStation_Feb2023/RapidTransitSystemStation.json','w') as f:
    f.write(json_dict)

In [8]:
with open ('static_data/TrainStation_Feb2023/RapidTransitSystemStation.json','r') as f:
    response = json.load(f)

# Bus Stops

In [58]:
from datamall_client import DataMallClient
from dotenv import load_dotenv
import os
load_dotenv()

True

In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
api_key = os.environ['DATA_MALL_API_KEY']
client = DataMallClient(api_key=api_key)

In [62]:
# 5090 bus stop in total
bus_stops = []
skip=0
while True:
    result = client.retrieve_bus_stops(skip=skip)['value']
    bus_stops.extend(result)
    if len(result)<500:
        # Means last page already so break
        break
    else:
        skip+=500
